BERT 모델 중에서 sentimental analasys와 관련된 모델을 찾아본 결과, 가장 다운로드 횟수가 많았던 nlptown/bert-base-multilingual-uncased-sentiment 모델을 가져왔습니다.

[허깅페이스 링크](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?text=%22As+I+deal+with+real+history+almost+as+it+is%2C+I+get+angry+while+watching+it%2C+but+I%27m+immersed+in+it+because+it%27s+interesting+in+the+movie.+Every+time+I+watch+it%2C+it+looks+new.+It%27s+a+work+that+has+both+meaning+and+fun)

In [1]:
import torch

In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

영화 리뷰를 파파고를 통해 영어로 번역했습니다

>"As I deal with real history almost as it is, I get angry while watching it, but I'm immersed in it because it's interesting in the movie. Every time I watch it, it looks new. It's a work that has both meaning and fun"

In [16]:
inputs = tokenizer("As I deal with real history almost as it is, I get angry while watching it, but I'm immersed in it because it's interesting in the movie. Every time I watch it, it looks new. It's a work that has both meaning and fun",
                   return_tensors="pt")

In [19]:
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'4 stars'

분석 결과, 1에서 5 레벨 중 긍정적인 감정의 레벨 4가 출력됐습니다

추가로 허깅페이스에서 트위터 감정 분석 데이터셋을 가져와 fine tuning하고, 예측까지 해보려 했지만 실패했습니다...

fine tuning 과정은 해당 페이지를 참고했습니다.

[허깅페이스 fine tuning](https://huggingface.co/docs/transformers/v4.32.0/ko/training)

In [15]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.6 MB/s eta 0:00:00


In [5]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset

dataset = load_dataset("carblacac/twitter-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for carblacac/twitter-sentiment-analysis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/carblacac/twitter-sentiment-analysis
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

In [10]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 660.6 kB/s eta 0:00:00


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'feeling'],
        num_rows: 119988
    })
    validation: Dataset({
        features: ['text', 'feeling'],
        num_rows: 29997
    })
    test: Dataset({
        features: ['text', 'feeling'],
        num_rows: 61998
    })
})

In [3]:
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].rename_column('feeling', 'label')

# 변경된 데이터셋을 확인합니다.
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 119988
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 29997
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 61998
    })
})


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", num_labels=5)

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.566583,0.720000
2,No log,0.828504,0.732000
3,No log,1.273314,0.743000


TrainOutput(global_step=375, training_loss=0.4041171875, metrics={'train_runtime': 375.7469, 'train_samples_per_second': 7.984, 'train_steps_per_second': 0.998, 'total_flos': 789354427392000.0, 'train_loss': 0.4041171875, 'epoch': 3.0})

In [15]:
trainer

In [43]:
dataset['train'][0]

{'text': '@fa6ami86 so happy that salman won.  btw the 14sec clip is truely a teaser',
 'label': 0}

In [45]:
data = {'text': ["As I deal with real history almost as it is, I get angry while watching it, but I'm immersed in it because it's interesting in the movie. Every time I watch it, it looks new. It's a work that has both meaning and fun"],
    'label' : [1]}

In [46]:
from datasets import Dataset

In [47]:
custom_dataset = Dataset.from_dict(data)

In [48]:
trainer.predict(custom_dataset)

ValueError: You have to specify either input_ids or inputs_embeds